In [1]:
## Librerias
import numpy as np
import scipy
from scipy import linalg
from numpy.linalg import inv

In [18]:
### La funcion descomponer recibe una matriz(lista de listas) de 3x4
### Devuelve las matrices K[3x3], R[3X3], t[3x1] 
def descomponer(listaDeListas):
    
    ## Convertimos lista de listas a matriz de Numpy
    matriz  = np.array(listaDeListas)
    print("\n Matriz M \n -------------")
    print(matriz)
    
    
    ## Calculamos S
    s = matriz[:,0:3]
    print("\n Matriz S \n -------------")
    print(s)
    
    
    ## Calculamos K y R utilizando rq de la libreria numpy.linalg
    k, r = linalg.rq(s)
    print("\n Matriz K \n -------------")
    print(k)
    
    print("\n Matriz R \n -------------")
    print(r)
    
    
    ## Cambiamos los signos de las columnas de K (cuando el elmento de su diagonal es menor a 0)
    ## Cambiamos los signos de las filas de R (cuando el elemento de la diagonal en K es menor a 0)
    for j in [0,1,2]:    
        if k[j,j] < 0:
            r[j,:] = -r[j,:]
            k[:,j] = -k[:,j]

    print("\n Matriz K sin negativos \n -------------")
    print(k)
    print("\n Matriz R sin negativos \n -------------")
    print(r)

    
    ## Calculamos w = k.t = tercer columna de M
    w = matriz[:,3]
    
    k_inv = inv(k)
    print("\n Matriz K invertida \n -------------")
    print(k_inv)
    
    t = k_inv.dot(w)
    print("\n Matriz t \n -------------")
    print(t)
    
    return k,r,t

In [25]:
## Prueba de la funcion descomponer

# Se crean matrices de k,r y t
k = [[1,2,0],[0,3,4],[0,0,2]]
r = [[0,0,1],[0,1,0],[1,0,0]]
t = [[1],[0],[2]]

rt = [[0,0,1,1],[0,1,0,0],[1,0,0,2]]

# Se crean las matrices en numpy
k  = np.array(k)
r  = np.array(r)
t  = np.array(t)
rt  = np.array(rt)

# Se calcula M
m = k.dot(rt)
print(m)

# Se descompone y se obtiene R, K y T para verificar si es igual a los creados anteriormente
k_calculada,r_calculada,t_calculada =  descomponer(m)


[[0 2 1 1]
 [4 3 0 8]
 [2 0 0 4]]

 Matriz M 
 -------------
[[0 2 1 1]
 [4 3 0 8]
 [2 0 0 4]]

 Matriz S 
 -------------
[[0 2 1]
 [4 3 0]
 [2 0 0]]

 Matriz K 
 -------------
[[-1.  2.  0.]
 [ 0.  3. -4.]
 [ 0.  0. -2.]]

 Matriz R 
 -------------
[[ 0.  0. -1.]
 [ 0.  1.  0.]
 [-1. -0.  0.]]

 Matriz K sin negativos 
 -------------
[[ 1.  2. -0.]
 [-0.  3.  4.]
 [-0.  0.  2.]]

 Matriz R sin negativos 
 -------------
[[-0. -0.  1.]
 [ 0.  1.  0.]
 [ 1.  0. -0.]]

 Matriz K invertida 
 -------------
[[ 1.         -0.66666667  1.33333333]
 [ 0.          0.33333333 -0.66666667]
 [ 0.          0.          0.5       ]]

 Matriz t 
 -------------
[ 1.  0.  2.]


In [29]:
### La funcion proyectar recibe una matriz con la posicion de dimensiones 3xN
### una matriz de calibracion de 3x4 y una matriz de 2x1 con el tamaño del sensor
### Devuelve la matriz proyeccion de [2xN]
def proyectar(pos, matCalib, tamanioSensor):
    ## Descomponemos M en Ma, Mb, Mc, que son sus filas
    ma = s = matCalib[:,0:1]
    

In [30]:
m = [[0,2,1,1],[4,3,0,8],[2,0,0,4]]
p = [[1,2,1],[2,1,1],[1,2,2]]
ta = [[1],[2]]

proyectar(p,m,ta)

TypeError: list indices must be integers, not tuple